## Requisitos

Si no lo tienes instalar, revise la documentacion para mas informacion [Docu](https://pypi.org/project/coboljsonifier/)
> pip install coboljsonifier

In [7]:
import json
from coboljsonifier.copybookextractor import CopybookExtractor
from coboljsonifier.parser import Parser
from coboljsonifier.config.parser_type_enum import ParseType
import pandas as pd
import ast

pd.options.display.max_rows = 4000

In [32]:
bookfname = 'bgdtacu.cob'
interface = 'BGDTACU-20230727.DAT'
rows = 2
# bin or ascii
tipo = 'bin' 

### Building a Parser

Use book structure to build a parser (**FLAT_ASCII** / **BINARY_EBCDIC**)

In [34]:
dict_structure = CopybookExtractor(bookfname).dict_book_structure

if tipo == 'bin':
    parser = Parser(dict_structure, ParseType.BINARY_EBCDIC).build()
    size = parser.size
    print("// Registry calculated lenght:", size)
if tipo == 'ascii':
    parser = Parser(dict_structure, ParseType.FLAT_ASCII).build()

// Registry calculated lenght: 243


In [23]:
df = pd.DataFrame([key for key in dict_structure.keys()], columns=['Name'])
df['type'] = [value['type'] for value in dict_structure.values()]
df['format'] = [value['format'] for value in dict_structure.values()]
df['size'] = [value['size'] for value in dict_structure.values()]
df['decimals'] = [value['decimals'] for value in dict_structure.values()]

In [3]:
#TODO: 2 Unir la implementacion para que todo quede en un solo nootbook 

In [26]:
with open(interface) as archivo:
    i = 1
    for linea in archivo:
        
        data= linea
        parser.parse(data)
        dictvalue = parser.value
        pretty = json.dumps(dictvalue, indent=4,default=str)
        val_convert = ast.literal_eval(pretty.replace("null", "None"))

        col = 'col' + str(i)
        df[col] = [value for value in val_convert.values()]

        if i==rows:
            break
          
        i = i + 1

In [27]:
df

,Name,type,format,size,decimals,col1,col2,col3
0,ALIC-TIPO-CBTE,NUMERIC,9(03),3,0,99,99,99
1,ALIC-PUNTO-VENTA,NUMERIC,9(05),5,0,7,7,7
2,ALIC-NRO-CBTE,ALPHANUMERIC,X(20),20,0,11000000062023040001,11000000072023040002,11000000102023040003
3,ALIC-COD-DOC-VEND,NUMERIC,9(02),2,0,80,80,80
4,ALIC-NRO-IDENT-VEND,ALPHANUMERIC,X(20),20,0,00000000030500003266,00000000030500001735,00000000030707202706
5,ALIC-SGN-NETO-GRAVADO,ALPHANUMERIC,X(1),1,0,0,0,0
6,ALIC-IMP-NETO-GRAVADO,NUMERIC,9(12)V99,14,2,3.65,109313.85,126.10
7,ALIC-ALICUOTA,NUMERIC,9(04),4,0,5,5,5
8,ALIC-SGN-IMPUESTO-IVA,ALPHANUMERIC,X(1),1,0,0,0,0
9,ALIC-IMP-IMPUESTO-IVA,NUMERIC,9(12)V99,14,2,0.77,22955.91,26.48


## Posibles errores

En **Extracting copybook structure** le da error los  |    PIC X.   => Cambiarlos por =>   PIC X(1).  en el copybook